Actividad #2 Maestría en ciencia de datos | Datos Masivos

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install langchain-huggingface

import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
import math


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [2]:
#Se crea el contexto de spark

#spark = SparkSession.builder.appName("Tarea 2 MCD DM").getOrCreate()
contexto = SparkContext("local", "Tarea 2 MCD DM")

Para la primer trabsformación de datos decidí usar un modelo para embeber texto, este modelo agarra un texto y dependiendo de su contenido genera 768 columnas donde se asigna un valor a cada columna dependiendo del contenido del texto.

In [3]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio

In [4]:
#Se especifica la ubicación del archivo donde se tienen los datos
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
#Se abre el archivo y se selecciona la columna que contiene los textos
%cd /content/drive/My Drive/
df = pd.read_csv('events.csv')
df_textos = df['text'].tolist()
df_test = df_textos[:2000]

/content/drive/My Drive


In [6]:
df_test[:15]

['Attempt missed. Mladen Petric (Hamburg) left footed shot from the left side of the box is high and wide to the left. Assisted by Gokhan Tore.',
 'Corner,  Borussia Dortmund. Conceded by Dennis Diekmeier.',
 'Corner,  Borussia Dortmund. Conceded by Heiko Westermann.',
 'Foul by Sven Bender (Borussia Dortmund).',
 'Gokhan Tore (Hamburg) wins a free kick in the defensive half.',
 'Hand ball by Jose Paolo Guerrero (Hamburg).',
 'Corner,  Hamburg. Conceded by Lukasz Piszczek.',
 'Chris Lowe (Borussia Dortmund) wins a free kick in the defensive half.',
 'Foul by Gojko Kacar (Hamburg).',
 'Foul by Gokhan Tore (Hamburg).',
 'Sven Bender (Borussia Dortmund) wins a free kick on the left wing.',
 'Attempt missed. Shinji Kagawa (Borussia Dortmund) right footed shot from outside the box is close, but misses the top right corner. Assisted by Mario Gotze.',
 'Foul by Gojko Kacar (Hamburg).',
 'Goal!  Borussia Dortmund 1, Hamburg 0. Kevin Grosskreutz (Borussia Dortmund) left footed shot from the lef

In [7]:
#Se genera el primer rdd y se le aplica una transformación para embeber los textos
count = 1
rdd = contexto.parallelize(df_test)
rdd_embedings = rdd.map(lambda x: embeddings_model.embed_query(x))


In [8]:
rdd_embed_col = rdd_embedings.collect()
#rdd_embed_col

In [10]:
#Se cuentan los elememtos del rdd para verificar que se hayan procesado los 2000 elementos
rdd_embed_col_2 = contexto.parallelize(rdd_embed_col)
rdd_embed_col_2.count()

2000

In [11]:
#Se muestra la información embebida del primer texto
first_embed = rdd_embed_col_2.take(1)
first_embed

[[0.03793489933013916,
  -0.047650136053562164,
  0.04716281220316887,
  -0.005998064763844013,
  0.014217648655176163,
  -0.05643584579229355,
  0.005961141083389521,
  -0.018665200099349022,
  -0.02980743534862995,
  0.040405839681625366,
  0.04378654807806015,
  -0.02868286333978176,
  0.015059683471918106,
  0.02553316205739975,
  0.06405650079250336,
  -0.024803906679153442,
  -0.005627556703984737,
  0.022419007495045662,
  -0.04843388497829437,
  0.042886558920145035,
  -0.035878028720617294,
  -0.0188464168459177,
  -0.04738921299576759,
  -0.006433161906898022,
  0.019672991707921028,
  0.009422135539352894,
  -0.05288128927350044,
  0.03126739338040352,
  -0.03457137197256088,
  -0.03276364877820015,
  0.017321307212114334,
  -0.04739063233137131,
  -0.049689386039972305,
  0.05897420272231102,
  1.674173063292983e-06,
  -0.03994134068489075,
  -0.010025222785770893,
  -0.03409915789961815,
  0.01938156597316265,
  -0.00034536674502305686,
  -0.019595732912421227,
  -0.027489

In [12]:
#Se cuentan los elementos de la informacion embebida del texto para verificar que nos haya generado las 768 columnas
first_embed_paral = contexto.parallelize(first_embed)
first_embed_paral_2 = first_embed_paral.flatMap(lambda x: x)
first_embed_paral_2.count()

768

Ahora con la columna de tipos de eventos se van a estandarizar los datos usando las operaciones de spark.

In [13]:
#Se obtiene la columna de tipo de evento
df_events = df['event_type'].tolist()

In [14]:
#Se genera el rdd para este segundo ejemplo
rdd2 = contexto.parallelize(df_events)

In [15]:
#Se obtiene la media de los tipos de eventos y se guarda.
rdd2_media = rdd2.sum()/rdd2.count()
rdd2_media

4.326574984936382

In [16]:
#Se calcula una parte de la formula de la desviación estandar: (valor actual - media)**2
rdd2_desvstd = rdd2.map(lambda x: (x-rdd2_media)**2)
rdd2_desvstd_col = rdd2_desvstd.collect()
rdd2_desvstd.take(5)

[11.06610113040449,
 5.412951160531727,
 5.412951160531727,
 1.7598011906589626,
 13.49405134129514]

In [17]:
#Una vez calculada la parte enterior se realiza lo faltante de la formula de la desviación estandar: sqrt(sum(valores calculados en rdd anterior)/cantidad de elementos)
#Se calcula la desviación estandar y se guarda
rdd2_desvstd_final = contexto.parallelize(rdd2_desvstd_col)
rdd2_desvstd_final_final = math.sqrt((rdd2_desvstd_final.sum()/rdd2_desvstd_final.count()))
rdd2_desvstd_final_final

2.9953118690011142

In [18]:
#Se estandarizan los datos

rdd2_estandarizado = rdd2.map(lambda x: (x-rdd2_media)/rdd2_desvstd_final_final)
rdd2_estandarizado_final = rdd2_estandarizado.collect()
rdd2_estandarizado.take(5)

[-1.1105938648204063,
 -0.7767388127474871,
 -0.7767388127474871,
 -0.4428837606745679,
 1.2263914996900283]

In [19]:
#Se cuenta con rdd2, la cantidad de datos con la que se trabajó:
rdd2.count()

941009